In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
df = pd.read_excel('Seasonality.xlsx')
# drops the extra title row
df.drop(df.index[0], axis=0, inplace=True)
# resets the index to start at 0. Could probably just make the datetime the index if you wanted.
df.reset_index(inplace=True, drop=True)

In [3]:
# supply an initial value to alpha
alpha = 0.3

In [4]:
# pulls the actual values into a numpy array
array_of_actual_values = df.Seasonally.values

In [5]:
def full_algorithim(alpha, init_data):
    # values predicted using the single-equation recursive form of Brown’s model
    y = np.empty(len(init_data), float)
    # error values (actual - predicted)
    z = np.empty(len(init_data), float)
    
    for i in range(0, len(init_data)):
        # sets the first two values in predicted to be the first value in actual
        if i < 2:
            y[i] = init_data[0]
        else:
            y[i] = 2 * init_data[i-1] - init_data[i-2] - 2 * (1-alpha) * z[i-1] + ((1-alpha)**2) * z[i-2]
            
        z[i] = init_data[i] - y[i]
        
    return np.sqrt(np.mean(np.square(z)))

In [7]:
result = minimize(full_algorithim, alpha, array_of_actual_values, bounds=[(0,1)])

In [8]:
# there's a tiny amount of variation in the optimal value based on the initial value for alpha
[alpha_opt] = result.x
print(alpha_opt)

0.47115429855026236


In [11]:
def algebraic_tranformation(df, columnNameActual, columnNamePredicted, columnNameErrors, alpha):
    df_new = df 
    
    for i in range(0, len(df_new)):
        if i < 2:
            # sets the first two values in Forecast to be the first value in Seasonally
            df_new.loc[i, columnNamePredicted] = df_new.loc[0, columnNameActual]
        else:
            df_new.loc[i, columnNamePredicted] = 2 * df_new.loc[i-1, columnNameActual] - df_new.loc[i-2, columnNameActual] - 2 * (1-alpha) * df_new.loc[i-1, columnNameErrors] + ((1-alpha)**2 * df_new.loc[i-2, columnNameErrors])
        # calculates the error, y_actual - y_predicted
        df_new.loc[i, columnNameErrors] = df_new.loc[i, columnNameActual] - df_new.loc[i, columnNamePredicted]
        
    return df_new

In [12]:
df_new = algebraic_tranformation(df, 'Seasonally', 'Forecast', 'Error', alpha_opt)

In [14]:
df_new.head()

,Date,Quarter,Original,Trend-Cycle,Ratio,Seasonal,Seasonally,Forecast,Error
0,1983-12-01,4.0,147.6,NaN,NaN,0.686136,215.118,215.117734,0.000000
1,1984-03-01,1.0,251.8,NaN,NaN,1.02386,245.931,215.117734,30.813656
2,1984-06-01,2.0,273.1,229.363,1.19069,1.1605,235.329,244.153707,-8.824883
3,1984-09-01,3.0,249.1,224.5,1.10958,1.1295,220.541,242.678156,-22.137602
4,1984-12-01,4.0,139.3,219.062,0.635892,0.686136,203.021,226.698911,-23.677906


In [19]:
# removes time from date column
df_new.Date = pd.to_datetime(df.Date).dt.date

with pd.ExcelWriter('Seasonality-with-Forecast-Errors.xlsx', engine='openpyxl', mode='w') as writer:
    df_new.to_excel(writer, index=False)